# PyTorch MNIST Membership Inference demo

This notebook contains an end-to-end demostration of Dioptra that can be run on any modern laptop.

## Setup

Below we import the necessary Python modules and ensure the proper environment variables are set so that all the code blocks will work as expected.

In [59]:
# Import packages from the Python standard library
import importlib.util
import os
import sys
import pprint
import time
import warnings
from pathlib import Path


def register_python_source_file(module_name: str, filepath: Path) -> None:
    """Import a source file directly.

    Args:
        module_name: The module name to associate with the imported source file.
        filepath: The path to the source file.

    Notes:
        Adapted from the following implementation in the Python documentation:
        https://docs.python.org/3/library/importlib.html#importing-a-source-file-directly
    """
    spec = importlib.util.spec_from_file_location(module_name, str(filepath))
    module = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = module
    spec.loader.exec_module(module)


# Filter out warning messages
warnings.filterwarnings("ignore")

# Experiment name
EXPERIMENT_NAME = "pytorch_mnist_mi"

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = "http://localhost:30080"

# Set DIOPTRA_RESTAPI_URI variable if not defined, used to connect to RESTful API service
os.environ["DIOPTRA_RESTAPI_URI"] = RESTAPI_ADDRESS

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = "http://localhost:35000"

# Set MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
if os.getenv("MLFLOW_TRACKING_URI") is None:
    os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Register the examples/scripts directory as a Python module
register_python_source_file("scripts", Path("..", "scripts", "__init__.py"))

from scripts.client import DioptraClient
from scripts.utils import make_tar

# Import third-party Python packages
import numpy as np
from mlflow.tracking import MlflowClient

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

## Submit and run jobs

The entrypoints that we will be running in this example are implemented in the Python source files under `src/` and the `src/MLproject` file.
To run these entrypoints within Dioptra's architecture, we need to package those files up into an archive and submit it to the Dioptra RESTful API to create a new job.
For convenience, we provide the `make_tar` helper function defined in `examples/scripts/utils.py`.

In [60]:
make_tar(["src"], WORKFLOWS_TAR_GZ)

PosixPath('/mnt/c/Users/jtsexton/Documents/GitHub/dioptra/examples/pytorch-maite/workflows.tar.gz')

To connect with the endpoint, we will use a client class defined in the `examples/scripts/client.py` file that is able to connect with the Dioptra RESTful API using the HTTP protocol.
We connect using the client below.
The client uses the environment variable `DIOPTRA_RESTAPI_URI`, which we configured at the top of the notebook, to figure out how to connect to the Dioptra RESTful API.

In [61]:
restapi_client = DioptraClient()

We need to register an experiment under which to collect our job runs.
The code below checks if the relevant experiment exists.
If it does, then it just returns info about the experiment, if it doesn't, it then registers the new experiment.

In [68]:
!python ../scripts/register_task_plugins.py --force --plugins-dir ../task-plugins --api-url http://localhost:30080

╭─────────────────────────────────────────────────╮
│ Dioptra Examples - Register Custom Task Plugins │
╰─────────────────────────────────────────────────╯
 ‣ plugins_dir: ../task-plugins
 ‣ api_url: http://localhost:30080
 ‣ force: True
 ✔ Overwritten. Removed and re-registered the custom task plugin 
'custom_fgm_plugins'.
 ✔ Overwritten. Removed and re-registered the custom task plugin 
'custom_patch_plugins'.
 ✔ Overwritten. Removed and re-registered the custom task plugin 
'custom_poisoning_plugins'.
 ✔ Overwritten. Removed and re-registered the custom task plugin 'evaluation'.
 ✔ Overwritten. Removed and re-registered the custom task plugin 
'feature_squeezing'.
 ✔ Overwritten. Removed and re-registered the custom task plugin 'maite'.
 ✔ Overwritten. Removed and re-registered the custom task plugin 
'model_inversion'.
 ✔ Overwritten. Removed and re-registered the custom task plugin 
'pixel_threshold'.
 ✔ Overwritten. Removed and re-registered the custom task plugin 'pytorch_d2'.
 

In [69]:
response_experiment = restapi_client.get_experiment_by_name(name=EXPERIMENT_NAME)

if response_experiment is None or "Not Found" in response_experiment.get("message", []):
    response_experiment = restapi_client.register_experiment(name=EXPERIMENT_NAME)

response_experiment

{'experimentId': 1,
 'createdOn': '2024-02-20T18:37:57.734596',
 'lastModified': '2024-02-20T18:37:57.734596',
 'name': 'pytorch_mnist_mi'}

Next, we need to train our model.
We will be using the V100 GPUs that are available on the DGX Workstation, which we can use by submitting our job to the `"tensorflow_gpu"` queue.
We will train one LeNet-5 model for use in the membership inference attack.


In [70]:
make_tar(["src"], WORKFLOWS_TAR_GZ)

PosixPath('/mnt/c/Users/jtsexton/Documents/GitHub/dioptra/examples/pytorch-maite/workflows.tar.gz')

In [71]:
response_test_metrics = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="full_workflow",
    entry_point_kwargs=" ".join([
        "-P dataset_name=cifar10",
        "-P subset=32"
    ]),
    queue="pytorch_cpu",
    timeout="1h",
)
pprint.pprint(response_test_metrics)

{'createdOn': '2024-02-22T16:37:12.028713',
 'dependsOn': None,
 'entryPoint': 'full_workflow',
 'entryPointKwargs': '-P dataset_name=cifar10 -P subset=32',
 'experimentId': 1,
 'jobId': 'b93a1578-3ae6-42de-8e74-fa293746a1fd',
 'lastModified': '2024-02-22T16:37:12.028713',
 'mlflowRunId': None,
 'queueId': 3,
 'status': 'queued',
 'timeout': '1h',
 'workflowUri': 's3://workflow/ebe69b0660944fe6835b5278cd90878b/workflows.tar.gz'}
